### Channel pruning (modified) 
...desmenuselo papi

In [1]:
from __future__ import print_function
from easydict import EasyDict as edict
from lib.cfgs import c as dcfgs
import lib.cfgs as cfgs
import os
os.environ['JOBLIB_TEMP_FOLDER']=dcfgs.shm
#import argparse
os.environ['GLOG_minloglevel'] = '3'
import os.path as osp
import pickle
import sys
from multiprocessing import Process, Queue

import matplotlib.pyplot as plt
import numpy as np
from IPython import embed

from lib.decompose import *
from lib.net import Net, load_layer, caffe_test
from lib.utils import *
from lib.worker import Worker
import google.protobuf.text_format

sys.path.insert(0, osp.dirname('__file__')+'/lib')

no lighting pack


In [3]:
def step0(pt, model):
    net = Net(pt, model=model, noTF=1) #lib/net.Net instantiate the NetBuilder
    WPQ, pt, model = net.preprocess_resnet()
    # WPQ stores pruned values, which will be later saved to the caffemodel.
    return {"WPQ": WPQ, "pt": pt, "model": model}

def step1(pt, model, WPQ, check_exist=False):
    print(pt)
    net = Net(pt, model, noTF=1)
    model = net.finalmodel(WPQ)
    if 1:
        convs = net.convs
    else:
        convs = net.convs[:-1]
        redprint("ignoring last conv!")
    if dcfgs.dic.option == 1:
        sums = net.type2names('Eltwise')[:-1]
        newsums = []
        for i in sums:
            if not i.endswith('block8_sum'):
                newsums.append(i)
        newconvs = []
        for i in convs:
            if i.endswith('_proj'):
                newconvs.insert(0,i)
            else:
                newconvs.append(i)
        convs = newsums + newconvs
    else:
        convs += net.type2names('Eltwise')[:-1]
    if dcfgs.dic.fitfc:
        convs += net.type2names('InnerProduct')
    if dcfgs.model in [cfgs.Models.xception,cfgs.Models.resnet]:
        for i in net.bns:
            if 'branch1' in i:
                convs += [i]
    net.freeze_images(check_exist=check_exist, convs=convs)
    return {"model":model}

def solve(pt, model):
    net = Net(pt, model=model)
    net.load_frozen()
    WPQ, new_pt = net.R3()
    return {"WPQ": WPQ, "new_pt": new_pt}  

def stepend(new_pt, model, WPQ):
    net = Net(new_pt, model=model)
    net.WPQ = WPQ
    net.finalmodel(save=False)
    net.dis_memory()
    #final = net.finalmodel(WPQ, prefix='3r')
    new_pt, new_model = net.save(prefix='3c')
    print('caffe test -model',new_pt, '-weights',new_model)
    return {"final": None}  

In [4]:
# ====== main() =======
cfgs.set_nBatches(dcfgs.nBatches)
dcfgs.dic.option=1

# asume action is c3
print("args.action = " , cfgs.Action.c3)

# ---- c3() ----
pt=cfgs.vgg.model
model=cfgs.vgg.weights
dcfgs.splitconvrelu=True
cfgs.accname='accuracy@5' #name of layer in the prototxt?
  
worker = Worker()
#print("before 'freezing'")
   
outputs = worker.do(step0, pt=pt, model=model)
printstage("freeze")
pt = outputs['pt']
print(outputs)

args.action =  c3
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
stage0 freeze
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
{'WPQ': {}, 'pt': 'temp/bn_vgg.prototxt', 'model': 'temp/bn_vgg.caffemodel'}


In [5]:
outputs = worker.do(step1,**outputs)
print(outputs)
printstage("speed ", dcfgs.dic.keep)

outputs['pt'] = mem_pt(pt)
print(outputs)

temp/bn_vgg.prototxt
nPicsPerBatch:  1
nPointsPerLayer 10
run for 500 batches nFeatsPerBatch 10
Extracting conv1_1 (5000, 64)
Extracting conv1_2 (5000, 64)
Extracting conv2_1 (5000, 128)
Extracting conv2_2 (5000, 128)
Extracting conv3_1 (5000, 256)
Extracting conv3_2 (5000, 256)
Extracting conv3_3 (5000, 256)
Extracting conv4_1 (5000, 512)
Extracting conv4_2 (5000, 512)
Extracting conv4_3 (5000, 512)
Extracting conv5_1 (5000, 512)
Extracting conv5_2 (5000, 512)
Extracting conv5_3 (5000, 512)
@ frozen_points is:  False
Acc  86.800
wrote memory data layer to temp/mem_bn_vgg.prototxt
freezing imgs to temp/frozen500.pickle
{'model': 'temp/VHmario_bn_vgg.caffemodel'}
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
stage1 speed 3.0
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
{'pt': 'temp/mem_bn_vgg.prototxt', 'model': 'temp/VHmario_bn_vgg.caffemodel'}


In [6]:
my_net = Net(pt, model=model)

In [7]:
my_net.load_frozen()

loading imgs from temp/frozen500.pickle
loaded


In [8]:
# === WPQ, new_pt = net.R3() ===
speed_ratio = dcfgs.dic.keep
if speed_ratio not in [3.]:
    NotImplementedError
if dcfgs.dic.vh:
    prefix = 'C'
else:
    prefix = 'noC'
prefix += str(int(speed_ratio))+'x'
DEBUG = True
convs= my_net.convs
my_net.WPQ = dict()
my_net.selection = dict()
my_net._mem = True
end = 5
alldic = ['conv%d_1' % i for i in range(1,end)] + ['conv%d_2' % i for i in range(3, end)]
pooldic = ['conv1_2', 'conv2_2']#, 'conv3_3']
rankdic = {'conv1_1': 17,
           'conv1_2': 17,
           'conv2_1': 37,
           'conv2_2': 47,
           'conv3_1': 83,
           'conv3_2': 89,
           'conv3_3': 106,
           'conv4_1': 175,
           'conv4_2': 192,
           'conv4_3': 227,
           'conv5_1': 398,
           'conv5_2': 390,
           'conv5_3': 379}

for i in rankdic:
    if 'conv5' in i:
        continue
    rankdic[i] = int(rankdic[i] * 4. / speed_ratio)
c_ratio = 1.15

In [9]:
def getX(name):
    x = my_net.extract_XY(my_net.bottom_names[name][0], name)
    return np.rollaxis(x.reshape((-1, 3, 3, x.shape[1])), 3, 1).copy()

def setConv(c, d):
    if c in self.selection:
        my_net.param_data(c)[:,my_net.selection[c],:,:] = d
    else:
        my_net.set_param_data(c, d)

In [98]:
#for conv, convnext in zip(convs[1:], convs[2:]+['pool5']):
#    print(conv, convnext)
convs

['conv1_1',
 'conv1_2',
 'conv2_1',
 'conv2_2',
 'conv3_1',
 'conv3_2',
 'conv3_3',
 'conv4_1',
 'conv4_2',
 'conv4_3',
 'conv5_1',
 'conv5_2',
 'conv5_3']

In [12]:
#conv = 'conv1_2'
#convnext = 'conv2_1'

run_once = True
t = Timer()

for conv, convnext in zip(convs[1:], convs[2:]+['pool5']):
    conv_V = underline(conv, 'V')
    conv_H = underline(conv, 'H')
    conv_P = underline(conv, 'P')
    W_shape = my_net.param_shape(conv)
    print("W_shape:", W_shape)
    d_c = int(W_shape[0] / c_ratio)
    print("d_c: ", d_c)
    rank = rankdic[conv]
    d_prime = rank
    if d_c < rank: d_c = rank

    '''spatial decomposition'''
    if False:
        t.tic()
        weights = self.param_data(conv)
        if conv in self.selection:
            weights = weights[:,self.selection[conv],:,:]
        if 1:
            Y = self._feats_dict[conv] - self.param_b_data(conv)
            X = getX(conv)
            if conv in self.selection:
                X = X[:,self.selection[conv],:,:]
            V, H, VHr, b = VH_decompose(weights, rank=rank, DEBUG=DEBUG, X=X, Y=Y)
            self.set_param_b(conv,b)
        else:
            V, H, VHr = VH_decompose(weights, rank=rank, DEBUG=DEBUG)


        self.WPQ[conv_V] = V

        # set W to low rank W, asymetric solver
        setConv(conv,VHr)

        self.WPQ[(conv_H, 0)] = H
        self.WPQ[(conv_H, 1)] = self.param_b_data(conv)
        if 0:#DEBUG:
            print("W", W_shape)
            print("V", V.shape)
            print("H", H.shape)

        t.toc('spatial_decomposition')

    #self.insert(conv, conv_H)

    '''channel decomposition'''
    if False:# and conv != 'conv3_3':
        t.tic()
        feats_dict, _ = self.extract_features(names=conv, points_dict=self._points_dict, save=1)
        Y = feats_dict[conv]
        W1, W2, B, W12 = ITQ_decompose(Y, self._feats_dict[conv], H, d_prime, bias=self.param_b_data(conv), DEBUG=0, Wr=VHr)

        # set W to low rank W, asymetric solver
        setConv(conv,W12.copy())
        self.set_param_b(conv, B.copy())

        # save W_prime and P params
        W_prime_shape = [d_prime, H.shape[1], H.shape[2], H.shape[3]]
        P_shape = [W2.shape[0], W2.shape[1], 1, 1]
        self.WPQ[(conv_H, 0)] = W1.reshape(W_prime_shape)
        self.WPQ[(conv_H, 1)] = np.zeros(d_prime)
        self.WPQ[(conv_P, 0)] = W2.reshape(P_shape)
        self.WPQ[(conv_P, 1)] = B

        self.insert(conv_H, conv_P, pad=0, kernel_size=1, bias=True, stride=1)

        t.toc('channel_decomposition')

        '''channel pruning'''
    if dcfgs.dic.vh and (conv in alldic or conv in pooldic) and (convnext in my_net.convs):
        t.tic()
        #if conv.startswith('conv4'):
        #    c_ratio = 1.5

        if conv in pooldic:
            X_name = my_net.bottom_names[convnext][0]
        else:
            X_name = conv

        idxs, W2, B2 = my_net.dictionary_kernel(X_name, None, d_c, convnext, None)
        """
        # W2
        self.selection[convnext] = idxs
        self.param_data(convnext)[:, ~idxs, ...] = 0
        self.param_data(convnext)[:, idxs, ...] = W2.copy()
        self.set_param_b(convnext,B2)
        # W1
        if (conv_P,0) in self.WPQ:
            key =  conv_P
        else:
            key = conv_H
        self.WPQ[(key,0)] = self.WPQ[(key,0)][idxs]
        self.WPQ[(key,1)] = self.WPQ[(key,1)][idxs]
        self.set_conv(key, num_output=sum(idxs))
        """
        t.toc('channel_pruning')
        if run_one:
            break

W_shape: (64, 64, 3, 3)
d_c:  55
Extracting X pool1 From Y conv2_1 stride 1


RuntimeError: set_input_arrays may only be called if the first layer is a MemoryDataLayer

In [ ]:
# setup H
H_params = {'bias':True}
H_params.update(self.infer_pad_kernel(self.WPQ[(conv_H, 0)], conv))
self.set_conv(conv_H, **H_params)
# setup V
V_params = self.infer_pad_kernel(self.WPQ[conv_V], conv)
self.set_conv(conv, new_name=conv_V, **V_params)
if 0:#DEBUG: 
    print("V", H_params)
    print("H", V_params)  

In [ ]:
new_pt = self.save_pt(prefix=prefix)